In [1]:
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter
import nltk

nltk.download('gutenberg')
!python -m spacy download en
%matplotlib inline

[nltk_data] Downloading package gutenberg to /Users/swang/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!

    Linking successful
    /Users/swang/anaconda3/lib/python3.6/site-packages/en_core_web_sm -->
    /Users/swang/anaconda3/lib/python3.6/site-packages/spacy/data/en

    You can now load the model via spacy.load('en')



Using supervised learning, predict whether a sentence comes from Alice in Wonderland or Persuasion

In [2]:
# The Bag of Words approach - generate counts of how many times a word appears as features

In [3]:
def text_cleaner(text):
    text = re.sub(r'--',' ', text) #this will remove double dahses -- from text and puts in space
    text = re.sub('[\[].*?[\]]', '', text) #this removes punctuation from text? replaces with nothing
       #doesn't seem logical to remove punc. not sure what this does
    text = ' '.join(text.split()) # text.split() splits all the words in the text into individual strings
    return text #' '.join() takes all individual strings inside a list and replaces with a space in between

In [4]:
# loading data
pers = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

In [5]:
pers = re.sub(r'Chapter \d+', '', pers)
alice = re.sub(r'CHAPTER .*', '', alice)
# cleaning the chapter titles because they will automatically give away their uniqueness

In [6]:
alice = text_cleaner(alice[:int(len(alice)/10)])
# using first 10 percent of the text. Alice[interger valuce (length of alice/10)]
pers = text_cleaner(pers[:int(len(pers)/10)])

In [7]:
# now parse the novels to create the tokens
nlp = spacy.load('en')
alice_doc = nlp(alice)
pers_doc = nlp(pers)

At this step, the nlp model has already tokenized all of our words

In [8]:
# group the sentences
alice_sents = [[sent, 'Carroll'] for sent in alice_doc.sents]
    #puts each sentence and adds Carroll to it
pers_sents = [[sent, 'Austen'] for sent in pers_doc.sents]
    #creates a set of sets

In [9]:
sentences = pd.DataFrame(alice_sents + pers_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


In [10]:
sentences.tail()

,0,1
412,"(But, it, was, not, a, merely, selfish, cautio...",Austen
413,"(Had, she, not, imagined, herself, consulting,...",Austen
414,"(The, belief, of, being, prudent, ,, and, self...",Austen
415,"(He, had, left, the, country, in, consequence, .)",Austen
416,"(A, few, months, had, seen, the, beginning, an...",Austen


In [11]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_ #also filtered out common root words into one
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words. 2000 of them?
    return [item[0] for item in Counter(allwords).most_common(2000)]

In [12]:
# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            print("Processing row {}".format(i))
            
    return df

In [13]:
# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(pers_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [14]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400


,dull,sense,labour,personal,properly,act,regarded,pride,well,christmas,...,heaven,capable,parliament,number,friendship,convince,equally,secondly,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


### Now you have a dataset that you can use any sort of machine learning classifier on

In [15]:
# Using a random forest classifier (sklearn's ensemble)
from sklearn import ensemble
from sklearn.model_selection import train_test_split

#create your model
rfc = ensemble.RandomForestClassifier(n_estimators=100)

#assign Y and X variables
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.996

Test set score: 0.8622754491017964


In [16]:
#Divergence between training and test scores is a sign of overfitting?

In [17]:
# Using logistic regression
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(penalty='l2') # No need to specify l2 as it's the default. But we put it for demonstration.
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(250, 1647) (250,)
Training set score: 0.976

Test set score: 0.8862275449101796


/Users/swang/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [18]:
# Using gradient boosting
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.968

Test set score: 0.8143712574850299


## Same trained model with new input of Jane Austen's Emma

In [19]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma[:int(len(emma)/60)])
print(emma[:100])

# Parse our cleaned data.
emma_doc = nlp(emma)

# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in pers_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to
Processing row 0
Processing row 50
Processing row 100
done


In [20]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.6330275229357798


col_0,Austen,Carroll
row_0,,
Austen,120,20
Carroll,60,18


Other modeling techniques (SVM?),
making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), 
making sentence-level features (number of words, amount of punctuation), 
or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), 

and anything else your heart desires. Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 90%.

Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work. This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.
Record your work for each challenge in a notebook and submit it below.

In [29]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words2(text, num):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_ #also filtered out common root words into one
                for token in text
                if not token.is_punct
                and not token.is_stop]
    allwords = [t.lower() for t in allwords]
    
    # Return the most common words. 2000 of them?
    return [item[0] for item in Counter(allwords).most_common(num)]

In [31]:
# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        words = [t.lower() for t in words]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            print("Processing row {}".format(i))
            
    return df

In [30]:
# Set up the bags.
alicewords = bag_of_words2(alice_doc, 1000)
persuasionwords = bag_of_words2(pers_doc, 1000)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [32]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400


,insufficient,retrench,dull,fact,way,sense,hold,kellynch,personal,properly,...,reduction,prompt,convince,friendship,mild,equally,russell,love,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


In [33]:
# Using a random forest classifier (sklearn's ensemble)
from sklearn import ensemble
from sklearn.model_selection import train_test_split

#create your model
rfc2 = ensemble.RandomForestClassifier(n_estimators=100)

#assign Y and X variables
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.988

Test set score: 0.8862275449101796


In [34]:
from sklearn import svm

svm = svm.SVC(gamma='scale')

In [35]:
trainsvm = svm.fit(X_train, y_train)

In [37]:
print(svm.score(X_train, y_train))
print(svm.score(X_test, y_test))

0.704
0.688622754491018
